In [ ]:
# Standard Python libs ----
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import os
import sys

# Add the parent directory to sys.path so src can be imported
# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

__file__ = "/data3/mjorge/LinkNodeCommunity"
sys.path.append(__file__)


import LinkNodeCommunity as linknode
from src.LinkNodeCommunity.utils import linknode_equivalence_partition
from src.viz.graphics import link_statistics_graph, linkcommunity_matrix_map, dendrogram

In [4]:
data = pd.read_table(
    "data/WikiVote/Wiki-Vote.txt", header=3, names=["source", "target"]
)

In [5]:
# Get unique nodes from both 'source' and 'target' columns
unique_nodes = pd.unique(data[['source', 'target']].values.ravel())

# Create a mapping from old node ids to new consecutive ids
node_mapping = {old_id: new_id for new_id, old_id in enumerate(sorted(unique_nodes))}

# Relabel the 'source' and 'target' columns using the mapping
data_relabel = data.replace({'source': node_mapping, 'target': node_mapping})

# data_relabel now has nodes relabeled from 0 to N-1

In [6]:
G = nx.from_pandas_edgelist(data_relabel, "source", "target", create_using=nx.DiGraph)
model = linknode.Clustering(G, similarity_index="cosine_similarity")
model.fit(use_parallel=True, flat_mode=True, method="edgelist", max_dist=1)

RuntimeError: Probability of being a tree (Ptree) was negative.

In [ ]:
import igraph as ig

# Convert NetworkX DiGraph to igraph Graph
edges = list(G.edges())
ig_G = ig.Graph(directed=True)
ig_G.add_vertices(G.number_of_nodes())
ig_G.add_edges(edges)

# Plot using spring layout
layout = ig_G.layout("fr")
ig.plot(ig_G, layout=layout, vertex_size=10, edge_arrow_size=0.5, bbox=(800, 800))